# Solvers ⚙️

In this exercise, you will investigate the effects of different `solvers` on `LogisticRegression` models.

👇 Run the code below to import the dataset

In [309]:
import pandas as pd

df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_dataset.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality rating
0,9.47,5.97,7.36,10.17,6.84,9.15,9.78,9.52,10.34,8.80,6
1,10.05,8.84,9.76,8.38,10.15,6.91,9.70,9.01,9.23,8.80,7
2,10.59,10.71,10.84,10.97,9.03,10.42,11.46,11.25,11.34,9.06,4
3,11.00,8.44,8.32,9.65,7.87,10.92,6.97,11.07,10.66,8.89,8
4,12.12,13.44,10.35,9.95,11.09,9.38,10.22,9.04,7.68,11.38,3


- The dataset consists of different wines 🍷
- The features describe different properties of the wines 
- The target 🎯 is a quality rating given by an expert

## 1. Target engineering

In this section, you are going to transform the ratings into a binary target.

👇 How many observations are there for each rating?

In [310]:
# YOUR CODE HERE
rating_counts = df['quality rating'].value_counts()


❓ Create `y` by transforming the target into a binary classification task where quality ratings below 6 are bad [0], and ratings of 6 and above are good [1]

In [311]:
# YOUR CODE HERE

df['binary target'] = df['quality rating'].apply(lambda x: 1 if x >= 6 else 0)

df['quality rating'] = df['binary target']
y = df['quality rating']
df


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality rating,binary target
0,9.47,5.97,7.36,10.17,6.84,9.15,9.78,9.52,10.34,8.80,1,1
1,10.05,8.84,9.76,8.38,10.15,6.91,9.70,9.01,9.23,8.80,1,1
2,10.59,10.71,10.84,10.97,9.03,10.42,11.46,11.25,11.34,9.06,0,0
3,11.00,8.44,8.32,9.65,7.87,10.92,6.97,11.07,10.66,8.89,1,1
4,12.12,13.44,10.35,9.95,11.09,9.38,10.22,9.04,7.68,11.38,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,6.93,4.49,8.25,8.60,9.41,11.07,8.38,10.89,12.42,8.99,1,1
99996,10.57,9.56,9.83,8.98,9.77,10.04,10.87,11.28,9.57,8.97,1,1
99997,10.23,10.98,11.74,11.76,8.87,9.03,9.93,9.86,10.04,8.66,0,0
99998,7.25,3.95,7.03,8.90,8.49,9.75,11.45,10.21,8.32,9.44,1,1


❓ Check the class balance of the new binary target

In [312]:
# YOUR CODE HERE
class_balance = df['quality rating'].value_counts(normalize=True)
class_balance


0    0.5001
1    0.4999
Name: quality rating, dtype: float64

❓ Create your `X` by normalising the features. This will allow for fair comparison of different solvers.

In [313]:
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
scaler = MinMaxScaler()

X = df.drop(['quality rating',"binary target"], axis=1) 


In [314]:

X = scaler.fit_transform(X)
scaler

MinMaxScaler()

## 2. LogisticRegression solvers

❓ Logistic Regression models can be optimized using different **solvers**. Make a comparison of the available solvers':
- Fit time - which solver is **the fastest**?
- Precision - **how different** are their respective precision scores?

Available solvers for Logistic Regression are `['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']`
 
For more information on these 5 solvers, check out [this Stack Overflow thread](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions)

In [315]:
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
import time
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

fit_times = {}
precisions = {}

for solver in solvers:
    start_time = time.time()
    model = LogisticRegression(solver=solver)
    model.fit(X_train, y_train)
    fit_time = time.time() - start_time
    y_pred = model.predict(X_test)
    precision = precision_score(y_test, y_pred)
    
    fit_times[solver] = fit_time
    precisions[solver] = precision

print("Fit Times:")
for solver, time_val in fit_times.items():
    print(f"{solver}: {time_val} seconds")

print("\nPrecisions:")
for solver, precision_val in precisions.items():
    print(f"{solver}: {precision_val}")

Fit Times:
newton-cg: 0.21616172790527344 seconds
lbfgs: 0.15663766860961914 seconds
liblinear: 0.18175101280212402 seconds
sag: 0.37571001052856445 seconds
saga: 0.5852327346801758 seconds

Precisions:
newton-cg: 0.880012433944669
lbfgs: 0.880012433944669
liblinear: 0.8806806391367503
sag: 0.880012433944669
saga: 0.880012433944669


In [316]:
# YOUR ANSWER
fastest_solver = "lbfgs"

<details>
    <summary>ℹ️ Click here for our interpretation</summary>

All solvers should produce similar precision scores because our cost-function is "easy" enough to have a global minimum which is found by all 5 solvers. For very complex cost-functions such as in Deep Learning, different solvers may stopping at different values of the loss function.

**The wine dataset**
    
If you check feature importance with sklearn's <a href="https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html">permutation_importance</a> on the current dataset, you'll see many features result in almost 0 importance. Liblinear solver successively moves only along *one* direction at a time, regularizing the others with L1 regularization (a.k.a, setting their beta to 0), which might provide a good fit for a dataset where many features are not that important in predicting the target.

❗️There is a cost to searching for the best solver. Sticking with the default (`lbfgs`) may save the most time overall, sklearn provides you this grid for an idea of which solver to choose to start off with: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers-chart.png" width=700>



</details> 

###  🧪 Test your code

In [317]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solvers',
    fastest_solver=fastest_solver
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/saikotdasjoy/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/saikotdasjoy/code/Saikot1997/data-solvers/tests
plugins: asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_solvers.py::TestSolvers::test_fastest_solver PASSED                 [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/solvers.pickle

git commit -m 'Completed solvers step'

git push origin master



## 3. Stochastic Gradient Descent

Logistic Regression models can also be optimized via Stochastic Gradient Descent.

❓ Evaluate a Logistic Regression model optimized via **Stochastic Gradient Descent**. How do its precision score and training time compare to the performance of the models trained in section 2?


<details>
<summary>💡 Hint</summary>

- If you are stuck, look at the [SGDClassifier doc](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)!

</details>



In [318]:
# YOUR CODE HERE
from sklearn.linear_model import SGDClassifier
# Create and fit the SGDClassifier model
model = SGDClassifier(loss='log', random_state=42)
start_time = time.time()
model.fit(X_train, y_train)
fit_time = time.time() - start_time

y_pred = model.predict(X_test)
precision = precision_score(y_test, y_pred)

print("SGDClassifier Results:")
print("Fit Time:", fit_time, "seconds")
print("Precision:", precision)


SGDClassifier Results:
Fit Time: 0.15158867835998535 seconds
Precision: 0.8854833630733704


/home/saikotdasjoy/.pyenv/versions/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


☝️ The SGD model should have one of the shortest times (maybe even shorter than `liblinear`), for similar performance. This is a direct effect of performing each epoch of the Gradient Descent on a single row as opposed to loading 100k rows into memory at a time.

## 4. Predictions

❓ Use the best model (balanced with short fit time and high precision) to predict the binary quality (0 or 1) of the following wine. Store your:
- `predicted_class`
- `predicted_proba_of_class` (i.e if your model predicted a class of 1 what is the probability it believes 1 to be the class should be between 0 and 1)

In [319]:
new_wine = pd.read_csv('https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers_new_wine.csv')
new_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol
0,9.54,13.5,12.35,8.78,14.72,9.06,9.67,10.15,11.17,12.17


In [320]:
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression

best_model = LogisticRegression(solver='lbfgs')
best_model.fit(X_train, y_train)
new_wine_scaled = scaler.transform(new_wine)
predicted_class = best_model.predict(new_wine_scaled)
predicted_proba_of_class = best_model.predict_proba(new_wine_scaled)[0][0]
predicted_proba_of_class

0.9680835386510153

# 🏁  Check your code and push your notebook

In [322]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'new_data_prediction',
    predicted_class=predicted_class,
    predicted_proba_of_class=predicted_proba_of_class
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/saikotdasjoy/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/saikotdasjoy/code/Saikot1997/data-solvers/tests
plugins: asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 2 items

test_new_data_prediction.py::TestNewDataPrediction::test_predicted_class PASSED [ 50%]
test_new_data_prediction.py::TestNewDataPrediction::test_predicted_proba PASSED [100%]

============================== 2 passed in 0.12s ===============================


💯 You can commit your code:

git add tests/new_data_prediction.pickle

git commit -m 'Completed new_data_prediction step'

git push origin master

